### Load Data

In [ ]:
import pandas as pd
import re
from datetime import datetime

courses_prices = pd.read_csv('./Data/course_prices.csv')
courses = pd.read_csv('./Data/Coursera_courses.csv')
users = pd.read_csv('./Data/Coursera_reviews.csv')
courses_metadata = pd.read_csv('./Data/CourseraDataset-Unclean.csv')
job_skills = pd.read_csv('./Data/job_skills.csv')
job_metadata = pd.read_csv('./Data/linkedin_job_postings.csv')
course_desc_1 = pd.read_csv('./Data/coursera_course_dataset_v3.csv')
course_desc_2 = pd.read_csv('./Data/coursera_courses (2).csv')

In [ ]:
courses

In [ ]:
users

In [ ]:
courses_metadata

In [ ]:
job_metadata

In [ ]:
job_skills

### Remove Duplicate Courses

In [ ]:
courses_metadata.rename(columns={'Course Title': 'name', 'Rating': 'Overall Ratings', 'Review': 'Num of Reviews', 'Offered By': 'institution'}, inplace = True)
courses_metadata.drop_duplicates(subset=['name'], inplace=True)

### Remove Duplicate Reviews

In [ ]:
users.drop_duplicates(subset=['reviews', 'reviewers', 'course_id'], inplace=True)

### Handle Null Values

In [ ]:
#print(job_skills.isnull().sum())
#print(courses.isnull().sum())
#print(users.isnull().sum())
job_skills = job_skills.dropna(subset=['job_skills'])

### Reviews Preprocessing

In [ ]:
import re

users['reviews'] = users['reviews'].astype(str)

# Remove rows where 'reviews' only contains punctuation
users = users[users['reviews'].str.contains(r'\w', regex=True)]

# Remove rows where 'reviews' contains emojis
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
users = users[~users['reviews'].apply(lambda x: bool(emoji_pattern.search(x)))]

# Remove rows where 'reviews' contains emails or websites
users = users[~users['reviews'].str.contains(r'\S*@\S*\s?|http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', regex=True)]
users


### Merge Courses with Users by course_id

In [ ]:
temp = pd.merge(users, courses, on="course_id", how="inner")
#temp.head()

### Merge Courses with additional dataset to get missing descriptions

In [ ]:
### course_desc_1
course_desc_1 = course_desc_1.loc[(course_desc_1['course_description'].notna())]
course_desc_1 = course_desc_1[['Title', 'Skills', 'course_description']]

### course_desc_2
course_desc_2 = course_desc_2.loc[(course_desc_2['course_description'].notna())]
course_desc_2 = course_desc_2[['course_title', 'course_skills', 'course_description']]

In [ ]:
description_temp = pd.merge(courses_metadata, course_desc_1, left_on="name", right_on="Title", how="left")
description_temp = description_temp.drop(columns=['Title'])
description_temp = pd.merge(description_temp, course_desc_2, left_on="name", right_on="course_title", how="left")
description_temp = description_temp.drop(columns=['course_title'])

### Merging descriptions

In [ ]:
def merge_description(row):

    x = row['What you will learn'] if pd.notna(row['What you will learn']) else ''
    y = row['course_description_y'] if pd.notna(row['course_description_y']) else ''
    z = row['course_description_x'] if pd.notna(row['course_description_x']) else ''
    
    if x == '' and y == '' and z == '':
        return ''
    if x == y:
        return x + ' ' + '\n' + ' ' + z
    elif x == z:
        return x + ' ' + '\n' + ' ' + y
    elif y == z:
        return y + ' ' + '\n' + ' ' + x
    else:
        return x + ' ' + '\n' + ' ' + y + ' ' + '\n' + ' ' + z

description_temp['description'] = description_temp.apply(merge_description, axis=1)
description_temp = description_temp.reset_index(drop=True)

### Merging Skills

In [ ]:
def merge_skills(row):

    x = row['Skill gain'] if pd.notna(row['Skill gain']) else ''
    y = row['Skills'] if pd.notna(row['Skills']) else ''
    z = row['course_skills'] if pd.notna(row['course_skills']) else ''
    
    if x == '' and y == '' and z == '':
        return ''
    if x == y:
        return x + ' , ' + z
    elif x == z:
        return x + ' , ' + y
    elif y == z:
        return z + ' , ' + x
    else:
        return x + ' , ' + y + ' , ' + z
    
description_temp['skills'] = description_temp.apply(merge_skills, axis=1)
description_temp = description_temp.reset_index(drop=True)

In [ ]:
description_temp = description_temp.drop(columns=['What you will learn', 'course_description_x', 'course_description_y', 'Skill gain', 'Skills', 'course_skills'])

### Remove non-ASCII and non-English rows in dataset

In [ ]:
def clean_text(text):
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Normalize text: lowercase and strip leading/trailing whitespace
    text = text.lower().strip()
    # Remove single and double quotation marks
    text = re.sub(r'[\[\]\'"]', '', text)
    # Remove parentheses
    text = re.sub(r'[()]', '', text)
    # Replace special characters and bullet points with empty string
    text = re.sub(r'[\t\n\r]+', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text

temp['reviewers'] = temp['reviewers'].apply(lambda x: x[3:])
temp['reviewers'] = temp['reviewers'].apply(clean_text)
temp['name'] = temp['name'].apply(clean_text)
temp['institution'] = temp['institution'].apply(clean_text)
temp['reviews'] = temp['reviews'].apply(lambda x: str(x))
temp['reviews'] = temp['reviews'].apply(clean_text)
temp['course_id'] = temp['course_id'].apply(clean_text)
print(temp)

### Preprocess course prices

In [ ]:
# courses_prices['name'] = courses_prices['name'].apply(clean_text)
# courses_prices['institution'] = courses_prices['institution'].apply(clean_text)
# courses_prices = courses_prices[courses_prices['is_english_name']]
# courses_prices.drop(columns=['is_english_name'], inplace=True)
# courses_prices.head()

### Preprocess courses metadata

In [ ]:
description_temp['skills'] = description_temp['skills'].apply(clean_text)
description_temp['Instructor'] = description_temp['Instructor'].apply(clean_text)
description_temp['institution'] = description_temp['institution'].apply(clean_text)
description_temp['Level'].fillna('None', inplace=True)
description_temp['name'] = description_temp['name'].apply(clean_text) 
description_temp['Duration'] = description_temp['Duration'].astype(str).str.extract('(\d+)').fillna(0).astype(int)
description_temp['Num of Reviews'] = description_temp['Num of Reviews'].astype(str).str.extract('(\d+)').fillna(0).astype(int)
description_temp = description_temp[description_temp['skills'] != '']
description_temp.head()

In [ ]:
# Filter out empty description rows
description_temp = description_temp[description_temp['description'].str.strip() != '']

# Filter out skills rows with just ","
description_temp = description_temp[~description_temp['skills'].str.strip().eq(',')]

# Reset the index
description_temp = description_temp.reset_index(drop=True)


### Preprocess jobs

In [ ]:
job_metadata['job_title'] = job_metadata['job_title'].apply(clean_text)
job_skills['job_skills'] = job_skills['job_skills'].apply(lambda x: str(x)).apply(clean_text)

### Merge Additional Metadata with Temp dataset

In [ ]:
courses_data = pd.merge(temp, description_temp, on=["name", "institution"], how="inner")

courses_data.shape

### Number of unique courses left

In [ ]:
unique_courses = courses_data.drop_duplicates(subset='name')
unique_courses = unique_courses.shape[0]
print(f"Data points: {courses_data.shape[0]}")
print(f"Unique courses: {unique_courses}")

### Convert Dates to Numerical Dates and Sort by Date

In [ ]:
def convert_to_numerical_date(text_date):
    actual_date = datetime.strptime(text_date, "%b %d, %Y")
    return actual_date.year, actual_date.month, actual_date.day

courses_data[['year', 'month', 'day']] = courses_data['date_reviews'].apply(lambda x: pd.Series(convert_to_numerical_date(x)))
courses_data['date'] = pd.to_datetime(courses_data[['year', 'month', 'day']])
courses_data = courses_data.sort_values(by='date')

### Filter Essential Columns in Courses Data

In [ ]:
final_courses = courses_data[['reviews', 'reviewers', 'rating', 'name', 'institution', 'Overall Ratings', 'Level', 'Duration', 'Num of Reviews', 'skills', 'Instructor', 'description', 'date']]
# final_courses.head()

In [ ]:
print(final_courses.columns)

### Merge Datasets regarding Jobs

In [ ]:
jobs_data = pd.merge(job_skills, job_metadata, on="job_link", how="inner")
#jobs_data.head()

### Filter Essential Columns in Jobs Data

In [ ]:
final_jobs = jobs_data[['job_skills', 'job_title', 'search_position', 'job_level']]
final_jobs.head()
unique_values = final_jobs['search_position'].unique()
print(unique_values)
# print(unique_values.tolist())

### Demean Ratings

In [ ]:
reviewer_average_rating = final_courses.groupby('reviewers')['rating'].transform('mean')
final_courses['Demeaned Rating'] = final_courses['rating'] - reviewer_average_rating 
# final_courses.head()

### Adding Price Column to Courses Data

In [ ]:
# final_courses = pd.merge(final_courses, courses_prices, on=['institution', 'name'], how='inner')
# # print(final_courses.isnull().sum())
# # print(final_courses.head())

### Remove empty Reviewers data

In [ ]:
final_courses = final_courses[final_courses['reviewers'] != '']

### Adding Popularity to Courses Data

In [ ]:
final_courses['Popularity'] = final_courses['Overall Ratings'] * final_courses['Num of Reviews']
# final_courses.head()

### Remove users with less than 3 interactions

In [ ]:
reviewer_counts = final_courses['reviewers'].value_counts()
reviewers_to_keep = reviewer_counts[reviewer_counts >= 3].index
final_courses = final_courses[final_courses['reviewers'].isin(reviewers_to_keep)]


In [ ]:
final_jobs

In [ ]:
final_courses

## EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Summary Statistics 

In [ ]:
print("Count of datapoints: ", final_courses.shape)
print("Count of Unique Reviewers: ", final_courses['reviewers'].nunique())
print("Count of Unique Courses: ", final_courses['name'].nunique())

In [ ]:
final_courses.describe().round(2)

### Histogram of Number of Ratings given by each Reviewer

In [ ]:
unique_reviewers = final_courses.groupby('reviewers').size().reset_index(name='count_of_reviews')

bin_interval = 200
bin_edges = np.arange(0, unique_reviewers['count_of_reviews'].max() + bin_interval, bin_interval)

ax = sns.histplot(data=unique_reviewers, x='count_of_reviews', bins=bin_edges, kde=True, edgecolor='black', stat='count')

for bin in ax.patches:
    if bin.get_height() > 0:
        ax.annotate(format(bin.get_height(), '.0f'), 
                    (bin.get_x() + bin.get_width() / 2., bin.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 5), 
                    textcoords = 'offset points')
    
plt.xlabel('Number of Reviews')
plt.ylabel('Frequency')
plt.title('Histogram of Number of Ratings given by each Reviewer')
plt.xlim(0, 1400)
plt.ylim(0, 25000)
plt.show()

### Explore Number of Reviews > 600 and Handle it 

In [ ]:
high_review_count = unique_reviewers[unique_reviewers['count_of_reviews'] > 600]
print(high_review_count)
unique_reviewers = unique_reviewers[unique_reviewers['count_of_reviews'] < 600]

### Remove Reviewers who have > 600 Number of Reviews from final_courses

In [ ]:
final_courses = final_courses[~final_courses['reviewers'].isin(high_review_count['reviewers'])]

### Plot New Histogram of Number of Ratings given by each Reviewer

In [ ]:
bin_interval = 25
bin_edges = np.arange(0, unique_reviewers['count_of_reviews'].max() + bin_interval, bin_interval)

ax = sns.histplot(data=unique_reviewers, x='count_of_reviews', bins=bin_edges, kde=True, edgecolor='black', stat='count')

for bin in ax.patches:
    if bin.get_height() > 0:
        ax.annotate(format(bin.get_height(), '.0f'), 
                    (bin.get_x() + bin.get_width() / 2., bin.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 5), 
                    textcoords = 'offset points')
    
plt.xlabel('Number of Reviews')
plt.ylabel('Frequency')
plt.title('New Histogram of Number of Ratings given by each Reviewer')
plt.xlim(0, 175)
plt.ylim(0, 25000)
plt.show()

### New Summary Statistics

In [ ]:
print("Count of datapoints: ", final_courses.shape)
print("Count of Unique Reviewers: ", final_courses['reviewers'].nunique())
print("Count of Unique Courses: ", final_courses['name'].nunique())

In [ ]:
final_courses.describe().round(2)

### Histogram of Popularity of each Course

In [ ]:
grouped_courses = final_courses.drop_duplicates(subset=['institution', 'name'])

bin_interval = 500
bin_edges = np.arange(0, grouped_courses['Popularity'].max() + bin_interval, bin_interval)

ax = sns.histplot(data=grouped_courses, x='Popularity', bins=bin_edges, kde=True, edgecolor='black', stat='count')

for bin in ax.patches:
    if bin.get_height() > 0:
        ax.annotate(format(bin.get_height(), '.0f'), 
                    (bin.get_x() + bin.get_width() / 2., bin.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 5), 
                    textcoords = 'offset points')
    
plt.xlabel('Popularity')
plt.ylabel('Frequency')
plt.title('Histogram of Popularity of each Course')
plt.xlim(0, 5000)
plt.ylim(0, 200)

plt.show()


### Histogram of Count of Ratings given for each Course

In [ ]:
grouped_courses = final_courses.drop_duplicates(subset=['institution', 'name'])

bin_interval = 100
bin_edges = np.arange(0, grouped_courses['Num of Reviews'].max() + bin_interval, bin_interval)

ax = sns.histplot(data=grouped_courses, x='Num of Reviews', bins=bin_edges, kde=True, edgecolor='black', stat='count')

for bin in ax.patches:
    if bin.get_height() > 0:
        ax.annotate(format(bin.get_height(), '.0f'), 
                    (bin.get_x() + bin.get_width() / 2., bin.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 5), 
                    textcoords = 'offset points')
    
plt.xlabel('Number of Ratings')
plt.ylabel('Frequency')
plt.title('Histogram of Number of Ratings for each Course')
plt.xlim(0, 1000)
plt.ylim(0, 210)

plt.show()

### Histogram of Overall Ratings of each Course


In [ ]:
grouped_courses = final_courses.drop_duplicates(subset=['institution', 'name'])

bin_interval = 1
bin_edges = np.arange(0, grouped_courses['Overall Ratings'].max() + bin_interval, bin_interval)

ax = sns.histplot(data=grouped_courses, x='Overall Ratings', bins=bin_edges, kde=True, edgecolor='black', stat='count')

for bin in ax.patches:
    if bin.get_height() > 0:
        ax.annotate(format(bin.get_height(), '.0f'), 
                    (bin.get_x() + bin.get_width() / 2., bin.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 5), 
                    textcoords = 'offset points')
    
plt.xlabel('Overall Ratings')
plt.ylabel('Frequency')
plt.title('Histogram of Overall Ratings of each Course')
plt.xlim(0, 5)
plt.ylim(0, 250)
plt.show()

### Histogram of Ratings given by Reviewers

In [ ]:
bin_interval = 1
bin_edges = np.arange(0, final_courses['rating'].max() + bin_interval, bin_interval)

ax = sns.histplot(data=final_courses, x='rating', bins=bin_edges, kde=True, edgecolor='black', stat='count')

for bin in ax.patches:
    if bin.get_height() > 0:
        ax.annotate(format(bin.get_height(), '.0f'), 
                    (bin.get_x() + bin.get_width() / 2., bin.get_height()), 
                    ha = 'center', va = 'center', 
                    xytext = (0, 5), 
                    textcoords = 'offset points')
    
plt.xlabel('Ratings')
plt.ylabel('Frequency')
plt.title('Histogram of Ratings given by each Reviewers')
plt.xlim(0, 5)
plt.ylim(0, 130000)
plt.show()

### Conclusion
1. Histogram of Overall Ratings for each Course and Histogram of Ratings given by each Reviewer is left skewed, implying that reviewers usually leave positive ratings. 
2. Histogram of Number of Ratings of each Course is right skewed, implying the possibility that not many reviewers would leave ratings. 

Overall, reviewers who leave a rating gave it a high score. There is a possibility that reviewers who are unhappy tend to not leave a rating. 

### Implications
1. It is hard to decipher what reviewers do not like since we do not have data regarding what they gave bad ratings for. 

### Current Solution
1. Use a demeaned rating where a demeaned rating > 0 implies that reviewers "like" the course while a demeaned rating <0 implies that reviewers "less like" the course. 

In [ ]:
final_courses.to_csv('final_courses.csv', index=False)
final_jobs.to_csv('final_jobs.csv', index=False)